<center> <img src = https://i.pinimg.com/736x/52/ed/5d/52ed5d7347c242b76c7e289e3ba8bbbe.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1046]:
import pandas as pd
import psycopg2

In [1047]:
# Параметры подключения
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [1048]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [1049]:
query_3_1 = f''' select count(distinct id) /*подсчет уникальных id вакансий*/
            from public.vacancies
'''
cursor = connection.cursor()
cursor.execute(query_3_1)
vacancies_count = cursor.fetchone()[0]

In [1050]:
print(f"Количество вакансий: {vacancies_count}")

Количество вакансий: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [1051]:
query_3_2 = f''' select count(id) /*подсчет id работодателей*/
        from public.employers
'''
cursor.execute(query_3_2)
employers_count = cursor.fetchone()[0]

In [1052]:
print(f"Количество работодателей: {employers_count}")

Количество работодателей: 23501


3. Посчитайте с помощью запроса количество регионов (таблица areas).

In [1053]:
query_3_3 = f''' select count(id) /*подсчет id регионов*/
    from public.areas
'''
cursor.execute(query_3_3)
areas_count = cursor.fetchone()[0]

In [1054]:
print(f"Количество регионов: {areas_count}")

Количество регионов: 1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица industries).

In [1055]:
query_3_4 = f''' select count(id) /*подсчет id сфер деятельности*/
    from public.industries
'''
cursor.execute(query_3_4)
industries_count = cursor.fetchone()[0]

In [1056]:
print(f"Количество сфер деятельности: {industries_count}")

Количество сфер деятельности: 294


***

 - Высокий спрос на рабочие места: почти 49 197 вакансий при 23 501 работодателе означает, что в среднем на одного работодателя приходится около 2,1 вакансии.
 - В России 1700 регионов, представлено 1362, то есть охват данных почти полный по стране.
 - Поскольку вакансий почти в 2 раза больше, чем работодателей, компании могут активно конкурировать за сотрудников.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [1057]:
query_4_1 = f''' select a.name area, count(v.id) cnt /* вывод названия региона и количества вакансий в этом регионе */
        from public.vacancies v
        left join public.areas a on v.area_id = a.id /* присоединение по id региона */
        group by a.name /* группировка по имени региона */
        order by 2 desc /*вывод в порядке возрастания по количеству id */
        limit 5 /* ограничение на вывод 1-ых 5-и*/
'''
cursor.execute(query_4_1)
df_area_cnt = cursor.fetchall()

In [1058]:
print(f"Количество ваканский в каждом регионе в порядке убвания;\nПриведены 5 реагионов с наибольшим количеством:\n{df_area_cnt}")

Количество ваканский в каждом регионе в порядке убвания;
Приведены 5 реагионов с наибольшим количеством:
[('Москва', 5333), ('Санкт-Петербург', 2851), ('Минск', 2112), ('Новосибирск', 2006), ('Алматы', 1892)]


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [1059]:
query_4_2 = f''' select count(id) /* подсчет id вакансий */
        from public.vacancies
        where salary_from is not null or salary_to is not null /* проверка на непустые значения зарплатной вилки */
'''
cursor.execute(query_4_2)
count_notempty = cursor.fetchone()[0]

In [1060]:
print(f"Количество вакансий с заполненным полем максимальной или минимальной зарплаты\t{count_notempty}")

Количество вакансий с заполненным полем максимальной или минимальной зарплаты	24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [1061]:
query_min_4_3 = f''' select round(avg(salary_from),0) /* средняя указанная нижняя зарплата */
            from public.vacancies
            where salary_from is not null /*проверка на учет непустых значений */
'''
cursor.execute(query_min_4_3)
av_salaryfrom = cursor.fetchone()[0]
query_max_4_3 = f''' select round(avg(salary_to),0) /*средняя верхняя указанная зарплата */
            from public.vacancies
            where salary_to is not null /*проверка на учет непустых значений */
'''
cursor.execute(query_max_4_3)
av_salaryto = cursor.fetchone()[0]

In [1062]:
print(f"Среднее значение минимальной ЗП {av_salaryfrom}")
print(f"Среднее значение максимальной ЗП {av_salaryto}")

Среднее значение минимальной ЗП 71065
Среднее значение максимальной ЗП 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [1063]:
query_4_4 = f''' select schedule, employment, count(id) /* вывод типа рабочего графика, типа трудоустройства, количство вакансий */
        from public.vacancies
        where schedule is not null and employment is not null /* проверка на непустые значения графика и трудоустройства */
        group by schedule, employment /* группировка по графику, потом по типу трудоустройства */
        order by 3 desc /* сортировка по количеству вакансий на убывание */
'''
cursor.execute(query_4_4)
df = cursor.fetchall()

In [1064]:
df

[('Полный день', 'Полная занятость', 35367),
 ('Удаленная работа', 'Полная занятость', 7802),
 ('Гибкий график', 'Полная занятость', 1593),
 ('Удаленная работа', 'Частичная занятость', 1312),
 ('Сменный график', 'Полная занятость', 940),
 ('Полный день', 'Стажировка', 569),
 ('Вахтовый метод', 'Полная занятость', 367),
 ('Полный день', 'Частичная занятость', 347),
 ('Гибкий график', 'Частичная занятость', 312),
 ('Полный день', 'Проектная работа', 141),
 ('Удаленная работа', 'Проектная работа', 133),
 ('Гибкий график', 'Стажировка', 116),
 ('Сменный график', 'Частичная занятость', 101),
 ('Удаленная работа', 'Стажировка', 64),
 ('Гибкий график', 'Проектная работа', 18),
 ('Сменный график', 'Стажировка', 12),
 ('Вахтовый метод', 'Проектная работа', 2),
 ('Сменный график', 'Проектная работа', 1)]

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [1065]:
query_4_5 = f''' select count(id), experience /* количество вакансий, опыт работы */
        from public.vacancies
        where experience is not null /* проверка на непустое значение опыта */
        group by experience /* группировка по указанному опыту */
        order by 1 asc /* сортировка по количеству вакансий в порядке возрастания */
'''
cursor.execute(query_4_5)
df = cursor.fetchall()

In [1066]:
df

[(1337, 'Более 6 лет'),
 (7197, 'Нет опыта'),
 (14511, 'От 3 до 6 лет'),
 (26152, 'От 1 года до 3 лет')]

***

- Москва и Санкт-Петербург лидируют с большим отрывом — это центры экономической активности. Минск и Алматы (столицы Беларуси и Казахстана) также в топе, что говорит о развитом рынке труда в этих городах.
- Разрыв между мин. и макс. ЗП (~40 тыс. руб.) указывает на широкий диапазон зарплат внутри одних и тех же позиций. Почти половина вакансий не указала зарплату — возможно, работодатели скрывают эту информацию или обсуждают её индивидуально.
- Полный день + полная занятость (35 367 вакансий) — абсолютный лидер. Удалённая работа + полная занятость (7802) — тренд на гибридные форматы. Классический офисный формат доминирует, но удалёнка сохраняет значительную долю (15-20% от топ-форматов). Гибкий график и частичная занятость менее востребованы, но представлены в нишах (например, стажировки).
- Рынок ориентирован на специалистов с опытом 1–6 лет. Вакансий для новичков (~7 тыс.) меньше, чем для опытных кандидатов, но их количество значимо. Высокий порог опыта (>6 лет) требуется редко — вероятно, такие роли закрываются внутри компаний.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [1067]:
query_5_1 = f''' select e.name, count(v.id) /* наименование компании, количество вакансий */
        from public.employers e
        left join public.vacancies v on v.employer_id = e.id /* присоединение по id работодателя */
        group by e.name /* группировка по имени работодателя */
        order by 2 desc /* сортировка по убыванию количества вакансий */
'''
cursor.execute(query_5_1)
df = cursor.fetchall()

In [1068]:
print(f"Первое место\t{df[0]}\nПятое место\t{df[4]}")

Первое место	('Яндекс', 1933)
Пятое место	('Газпром нефть', 331)


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [1069]:
query_5_2 = f''' select a.name name_area, /* наименование региона */
        count(v.id) count_vacancies, /* количество вакансий в нем */
        count(e.id) count_employer /* количество работодателей в регионе */
        from public.areas a
        left join public.vacancies v on v.area_id = a.id /* присоединение таблицы с вакансиями по id региона */
        left join public.employers e on e.area = a.id /* присоединение таблицы с работодателями по id региона */
        group by a.name /* группировка по имени региона */
        having count(v.id) = 0 and count(e.id) > 0 /* условие на регионы с отуствием вакансий, но наличием работодателей */
        order by 3 desc /* сортировка по количеству работодателей в порядка убывания */
        limit 1 /* ограничение на вывод 1 строки */
'''
cursor.execute(query_5_2)
result = cursor.fetchall()

In [1070]:
print(f"Регион с отсутствием вакансий, но с наибольшим количеством работодателей {result[0]}")

Регион с отсутствием вакансий, но с наибольшим количеством работодателей ('Россия', 0, 410)


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [1071]:
query_5_3 = f''' select e.name, count(distinct v.area_id) /* вывод наименования работодателя, количество уникальных id региона */
        from public.vacancies v
        left join public.employers e on e.id = v.employer_id /* присоединение по id работодателя */
        group by e.name /* группировка по наименованию работодателя */
        order by 2 desc /* сортировка в порядке убывания по количеству id регионов */
'''
cursor.execute(query_5_3)
result = cursor.fetchall()

In [1072]:
result

[('Яндекс', 181),
 ('Ростелеком', 152),
 ('Спецремонт', 116),
 ('Поляков Денис Иванович', 88),
 ('ООО ЕФИН ', 71),
 ('Совкомбанк', 63),
 ('МТС', 55),
 ('ЭФКО, Управляющая компания', 49),
 ('Почта России', 48),
 ('КРОН', 48),
 ('ИК СИБИНТЕК', 46),
 ('MCORE', 46),
 ('Сбербанк-Сервис', 45),
 ('ANCOR', 44),
 ('Тинькофф', 43),
 ('Первый Бит', 43),
 ('АЛНАС', 41),
 ('Ozon', 36),
 ('IBS', 36),
 ('Авито', 35),
 ('Т1 Интеграция', 34),
 ('АТМ АЛЬЯНС', 34),
 ('ЭР-Телеком', 33),
 ('ЛАНИТ', 29),
 ('1C-Рарус', 28),
 ('Красное & Белое, розничная сеть', 27),
 ('Volna.tech', 27),
 ('Bell Integrator', 26),
 ('Совкомбанк Технологии', 26),
 ('Спортмастер', 25),
 ('МАГНИТ, Розничная сеть', 25),
 ('Softline', 25),
 ('Playrix', 25),
 ('Тензор', 25),
 ('Филиал ФКУ Налог-Сервис ФНС России по ЦОД в г.Москве', 25),
 ('СБЕР', 24),
 ('Нэти', 24),
 ('Газпром нефть', 24),
 ('Банк ВТБ (ПАО)', 24),
 ('МегаФон', 23),
 ('Диалог Регионы', 23),
 ('Сеть магазинов цифровой и бытовой техники DNS', 23),
 ('«Аксиома-Софт»', 23

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [1073]:
query_5_4 = f''' select count(distinct e.id) /* подсчет количества уникальных id работодателей */
        from public.employers e
        left join public.employers_industries ei on ei.employer_id = e.id /* присоединение сфер деятельности по id работодателя */
        left join public.industries i on i.id = ei.industry_id /* присоединение таблицы сфер деятельности по id сферы деятельности */
        where i.name is null /* проверка на непустое наименование сферы деятельности */
'''
cursor.execute(query_5_4)
result = cursor.fetchone()

In [1074]:
print(f"Количество работодателей, у которых не указана сфера деятельности {result[0]}")

Количество работодателей, у которых не указана сфера деятельности 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [1075]:
query_5_5 = f''' select e.name /* вывод наименования компании */
        from public.employers e
        left join public.employers_industries ei on ei.employer_id = e.id /* присоединение сфер деятельности по id компании */
        left join public.industries i on i.id = ei.industry_id /* присоединение сфер деятельности по id сферы деятельности */
        group by e.name /* группировка по наименованию компании */
        having count(distinct ei.industry_id) = 4 /* условие на наличие 4 уникальных сфер деятельности в описании компании */
        order by 1 asc /* сортировка по наименованию компании в алфавитном порядке */
        offset 2 /* пропуск первых двух строк в выводе */
        limit 1 /* граничение по выводу 1 строки */
'''
cursor.execute(query_5_5)
result = cursor.fetchall()

In [1076]:
print(f"Название компании на 3ем месте в алфавитном списке {result[0]}")

Название компании на 3ем месте в алфавитном списке ('2ГИС',)


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [1077]:
query_5_6 = f''' select count(distinct e.id) /* количество уникальных id работодателя */
        from public.employers e
        left join public.employers_industries ei on ei.employer_id = e.id /* присоеднинение сфер деятельности по id компании */
        left join public.industries i on i.id = ei.industry_id /* присоединение сфер деятельности по id работодателя */
        where i.name = 'Разработка программного обеспечения' /* условие на наименование сферы деятельности */
'''
cursor.execute(query_5_6)
count = cursor.fetchone()[0]

In [1078]:
print(f"Количество работодателей, у которых в сфере деятельности «Разработка программного обеспечения» {count}")

Количество работодателей, у которых в сфере деятельности «Разработка программного обеспечения» 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [1079]:
# Импорт библиотеки для обращения с странице по url запроса 
import requests
# Импорт библиотеки для излвечения текста с запроса страницы
from bs4 import BeautifulSoup 
# Ссылка на страницу с городами-милионниками
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
# Создание запроса
response = requests.get(url)
# Извлечение текста со страницы запроса
page = BeautifulSoup(response.text, 'html.parser')
# Ищем текст по тегу a. Тег a выбран, так как в html-коде страницы нужный список городов под тегом </a>
print(page.find('a'))
links = page.find_all('a')
# ResultSet ведет себя как список, но содержит объекты Tag, а не строки
texts = [tag.get_text() for tag in links]
# Выбираем индексы городов из начала и конца списка, представленного на странице wiki
index1 = texts.index('Москва')
index2 = texts.index('Волгоград')
# Выводим список городов от Москвы до Волгограда
print([link.text for link in links[index1:index2+1]])
# Длина списка
print(len([link.text for link in links[index1:index2+1]]))
# Присваем список переменной
list_cities = [link.text for link in links[index1:index2+1]]
# Поскольку SQL работает с f-строкой, нужно преобразовать данные в f-строку
cities_sql = ", ".join([f"'{city}'" for city in list_cities])

<a id="top"></a>
['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']
16


In [1080]:
query_5_7 = f''' (select a.name name_area, count(v.id) vacancy_count /* наименование региона, подсчет количества id вакансий */
        from public.areas a
        left join public.vacancies v on v.area_id = a.id /* присоединение по id региона */
        left join public.employers e on e.id = v.employer_id /* присоединение по id работодателя */
        where e.name = 'Яндекс' and a.name in ({cities_sql}) /* формат вставки списка, который воспринимает SQL */
        group by a.name /* группировка по наименованию региона */
        order by 2 desc) /* сортировка в порядке убывания по количеству вакансий */
        union all
        (select 'total', count(v.id) vacancy_count /* подсчет количества id вакансий */
        from public.areas a
        left join public.vacancies v on v.area_id = a.id /* присоединение по id региона */
        left join public.employers e on e.id = v.employer_id /* присоединение по id работодателя */
        where e.name = 'Яндекс' and a.name in ({cities_sql})) /* условие на наименование компании и наличие региона в списке cities_sql */
'''
cursor.execute(query_5_7)
result = cursor.fetchall()

In [1081]:
result

[('total', 485),
 ('Москва', 54),
 ('Санкт-Петербург', 42),
 ('Екатеринбург', 39),
 ('Нижний Новгород', 36),
 ('Новосибирск', 35),
 ('Воронеж', 32),
 ('Краснодар', 30),
 ('Самара', 26),
 ('Уфа', 26),
 ('Казань', 25),
 ('Пермь', 25),
 ('Ростов-на-Дону', 25),
 ('Волгоград', 24),
 ('Красноярск', 23),
 ('Челябинск', 22),
 ('Омск', 21)]

***

- Доминирование IT-гигантов (Яндекс, Tinkoff) и госкорпораций (Ростелеком, Газпром).-
- Проблемы с данными: многие работодатели не указывают сферу деятельности; регионы с работодателями, но без вакансий, требуют проверки.
- Крупные IT-компании активно нанимают в городах-миллионниках, не только в Москве.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [1082]:
query_6_1 = f''' select count(name) /* подсчет количества наименования вакансий */
        from public.vacancies
        /* для регистронезависимого поиска вместо like используем ilike */
        where name ilike '%data%' or name ilike '%данн%' /* условие на наличие в наименовании вакансии 'data' и 'данн' */
'''
cursor.execute(query_6_1)
result = cursor.fetchone()[0]

In [1083]:
result

1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [1084]:
query_6_2 = f''' select count(distinct id) /*  подсчет уникального количества id вакансий */
        from public.vacancies
        where (name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%'
        or (name ilike '%ml%' and name not ilike '%HTML%') or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
        and /* условие поиска на DS специалиста */
        (name ilike '%junior%' or experience = 'Нет опыта' or employment = 'Стажировка') /* условие поиска на junior-специалиста */
'''
cursor.execute(query_6_2)
result = cursor.fetchone()[0]

In [1085]:
result

51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [1086]:
query_6_3 = f''' select count(id) /* подсчет id вакансий */
        from public.vacancies
        where (key_skills ilike '%sql%' or key_skills ilike '%postgres%' and key_skills is not null) /* условие на ключевые навыки */
        and
        /* условие на DS специалиста */
        (name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%'
        or (name ilike '%ml%' and name not ilike '%HTML%') or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
'''
cursor.execute(query_6_3)
result = cursor.fetchone()[0]

In [1087]:
result

229

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [1088]:
query_6_4 = f''' select count(id) /* подсчет количества вакансий */
        from public.vacancies
        where (key_skills ilike '%Python%' and key_skills is not null) /* условие на ключевые навыки */
        and
        /* условие на DS специалиста */
        (name ilike '%data scientist%' or name ilike '%data science%' or name ilike '%исследователь данных%'
        or (name ilike '%ml%' and name not ilike '%HTML%') or name ilike '%machine learning%' or name ilike '%машинн%обучен%')
'''
cursor.execute(query_6_4)
result = cursor.fetchone()[0]

In [1089]:
result

357

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [1090]:
query_6_5 = f''' select 
/* вычисляем среднее от длины строки в ключевыми навыками (табуляцию заменяем на пробел) */
round(avg(char_length(key_skills) - char_length(replace(key_skills, chr(9), '')) + 1), 2) as cnt_w
from vacancies
where
    /* условие на поиск вакансии для DS специалистов */
    (lower(name) like '%data scientist%' or
    lower(name) like '%data science%' or
    lower(name) like '%исследователь данных%' or
    (name like '%ML%' and lower(name) not like'%html%') or
    lower(name) like '%machine learning%' or
    lower(name) like '%машинн%обучен%') and
    /* условие на ключевые навыки */
    (key_skills is not null and key_skills != '')
'''
cursor.execute(query_6_5)
result = cursor.fetchone()[0]

In [1091]:
result

Decimal('6.41')

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [1092]:
query_6_6 = f''' select 
    experience, /* вывод указанного опыта работы */
    /* округление среднего между нижним и верхним уровнем указанной ЗП */
    round(avg((coalesce(salary_to, salary_from) + coalesce(salary_from, salary_to))/2), 0) as avg_salary
    from 
    public.vacancies
    where 
    /* условие на нименовании вакансии для DS специалиста */
    (lower(name) like '%data scientist%' or
     lower(name) like '%data science%' or
     lower(name) like '%исследователь данных%' or
     (name like '%ML%' and lower(name) not like '%html%') or
     lower(name) like '%machine learning%' or
     lower(name) like '%машинн%обучен%') and
    (salary_to is not null or salary_from is not null)
    group by 
    experience /* группировка по опыту работы */
 '''
cursor.execute(query_6_6)
result = cursor.fetchall()

In [1093]:
result

[('Нет опыта', Decimal('74643')),
 ('От 1 года до 3 лет', Decimal('139675')),
 ('От 3 до 6 лет', Decimal('243115'))]

***

- Python доминирует — его упоминают в каждой 5-й DS-вакансии. Без него вход в профессию почти невозможен.
- Junior-позиций мало (51), а зарплаты для новичков низкие (~75 тыс.). Это сигнал: начинающим стоит брать стажировки или пет-проекты для портфолио.
- Опыт окупается: Senior DS (3–6 лет) получает 243 тыс. руб. — в 3.3 раза больше Junior.

# Общий вывод по проекту

**1. Ключевые выводы**

1.1. Общий рынок вакансий

Всего вакансий: ~49 тыс., из них 1 771 относятся к Data Science.
- География:\
Лидеры — Москва (5 333), Санкт-Петербург (2 851), Минск (2 112).\
Яндекс — самый активный работодатель (1 933 вакансии, 181 регион).
- Зарплаты:\
Средняя: 71–110 тыс. руб. (мин./макс.), но только 50% вакансий указывают ЗП.\
Для DS: 74 тыс. (Junior) → 243 тыс. (Senior 3–6 лет).

1.2. Data Science

*Начинающие (Junior):*\
Всего 51 вакансия (мало!); средняя ЗП — 74.6 тыс. руб.\
Конкуренция высокая: требуется портфолио или стажировки.\
Ключевые навыки:\
Python (20% вакансий), SQL (13%), ML (по ключевым словам).\
В среднем 6.4 навыка на вакансию — работодатели ждут «универсалов».

*Опыт vs ЗП:*\
1–3 года: +87% к ЗП (139.7 тыс.).\
3–6 лет: +74% к предыдущему уровню (243.1 тыс.).

1.3. Работодатели и форматы работы\
Топ-5 работодателей: Яндекс, Газпром нефть, Ростелеком, Сбер, Tinkoff.\
Форматы:\
Полный день (35 тыс. вакансий) доминирует, но удалёнка сохраняет 15–20%.\
Проблемы данных:\
8 419 работодателей не указали сферу деятельности.

**2. Прогнозы и тренды**

1. Рост удалённых ролей:
Уже сейчас 7.8 тыс. вакансий — удалёнка. Через 2–3 года может вырасти до 25–30%.
2. Спрос на Senior DS:
При текущем разрыве в ЗП (243 тыс. vs 74 тыс.) компании будут конкурировать за опытных специалистов.
3. Сужение входа для Junior:
Если Junior-ролей всего 51, новичкам придется активнее прокачивать hard/soft skills.

# Дополнительный анализ

1. Вывод топ-10 востребованных навыков. Что еще помимо Python и SQL для DS специалистов?

In [1094]:
query_1 = f''' with skill_unnested as ( /* удаляем пробелы с начала и конца строки, массив -> столбец, строка -> массив с разделителем ',' */ 
        select trim(unnest(string_to_array(key_skills, ','))) as skill
        from public.vacancies
        where key_skills is not null) /* условие на непустое поле ключевых навыков */
        select skill, count(*) as demand_count /* количество навыков */
        from skill_unnested
        group by skill /* группировка по навыкам */
        order by demand_count desc /* сортировка по количеству навыков по убыванию */
        limit 10 /* вывод первых 10-и */
'''
cursor.execute(query_1)
result = cursor.fetchall()

In [1095]:
result

[('1С программирование', 94),
 ('ответственность', 87),
 ('Пользователь ПК\tРабота в команде\tРемонт ноутбуков\tДиагностика\tОпытный пользователь ПК\tРемонт\tУстановка ОС\tРемонт компьютеров\tНавыки продаж',
  83),
 ('PostgreSQL\tSQL\tLinux\t1С программирование\tMS SQL', 71),
 ('Ответственность\tКлиентоориентированность', 48),
 ('Excel', 43),
 ('стрессоустойчивость', 28),
 ('Java', 28),
 ('MS PowerPoint\tПодготовка презентаций\tРазработка регламентов\tMS Visio\tМоделирование бизнес процессов\tОптимизация бизнес процессов\tАналитическое мышление',
  27),
 ('Настройка ПК\tСборка ПК\tРемонт ПК\tНастройка ПО\tНастройка сетевых подключений\tТехническое обслуживание\tТочность и внимательность к деталям\tСистемы видеонаблюдения\tАдминистрирование\tОфисная техника\tОпытный пользователь ПК',
  26)]

2. Какие отрасли больше всего нанимают (топ-6)?

In [1096]:
query_2 = f''' select i.name as industry, /* вывод наименования деятельности */
        count(v.id) as vacancies_count /* вывод количества id вакансий */
        from
        public.vacancies v
        left join public.employers_industries ei on v.employer_id = ei.employer_id /* присоединение по id компании */
        left join industries i on ei.industry_id = i.id /* присоединение по id отрасли */
        group by i.name /* группировка по наименованию отрасли */
        order by vacancies_count desc /* сортировка по количеству вакансий в порядке убывания */
        limit 6 /* ограничение на вывод 6-и */
'''
cursor.execute(query_2)
result = cursor.fetchall()

In [1097]:
result

[('Разработка программного обеспечения', 12499),
 ('Системная интеграция,  автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг',
  11034),
 (None, 10084),
 ('Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее)',
  6413),
 ('Банк', 2742),
 ('Мобильная связь', 1585)]

3. Отрасли с самыми высокими средними (между максимальной и минимальной указанными) ЗП (топ-6)

In [1098]:
query_3 = f''' select i.name as industry, /* вывод наименования деятельности */
        round(avg((coalesce(v.salary_from,0) + coalesce(v.salary_to,0))/2),0) as avg_salary, /* вывод средней ЗП */
        count(v.id) as vacancies_count /* количество вакансий */
        from public.vacancies v
        join employers_industries  ei on v.employer_id = ei.employer_id /* присоединение по id компании */
        join public.industries i on ei.industry_id = i.id /* присоединение по id отрасли */
        where v.salary_from is not null or v.salary_to is not null /* условие на непустые значения нижней и верхней указанной ЗП */
        group by i.name /* группировка по наименованию отрасли */
        order by avg_salary desc /* сортировка по средней ЗП */
        limit 6 /* ограничение на вывод 6 */
 '''
cursor.execute(query_3)
result = cursor.fetchall()

In [1099]:
result

[('Агрохимия (продвижение, оптовая торговля)', Decimal('190714'), 14),
 ('Агрохимия (производство)', Decimal('171813'), 16),
 ('Корма для животных (производство)', Decimal('158040'), 29),
 ('Зоомагазин', Decimal('152935'), 31),
 ('Корма для животных (продвижение, оптовая торговля)', Decimal('104381'), 57),
 ('Учреждение соц.помощи и защиты', Decimal('99844'), 16)]

# Общий вывод по дополнительному анализу

1. Востребованные hard- и soft-skills:
- hard: 1С-программирование, Excel;
- soft: ответственность и стрессоустойчивость.
2. Больше всего вакансий от IT-компании и финансов/телекоммуникаций.
3. Лидеры по средней ЗП - отрасли по работе с животными и детьми; вакансий мало, этим и объясняется доминирование.

In [1100]:
connection.close()